In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Simplex Tableau 

In [3]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import copy
import sympy as sp
from sympy import sympify
import ipysheet
from ipysheet import sheet, cell
from ipysheet import column, row
from IPython.display import clear_output
from IPython.display import Javascript
import ipython_blocking
import simplex_algorithm as sa

M = sp.symbols('M')



In [4]:


def on_restr_change(change):
    with out:
        if change['new'] != '0':
            anz_var.options = list(map(str, range(1, int(change['new'])+1,)))
            anz_schlupf_var.options = list(map(str, range(0, int(change['new'])+1)))
            anz_kuenstl_var.options = list(map(str, range(0, int(change['new'])+1)))
            anz_var.layout.visibility = "visible"
            anz_schlupf_var.layout.visibility = "visible"
            anz_kuenstl_var.layout.visibility = "visible"
            button.layout.visibility = "visible"
        else:
            anz_var.layout.visibility = "hidden"
            anz_schlupf_var.layout.visibility = "hidden"
            anz_kuenstl_var.layout.visibility = "hidden"


def create_tableau(button): 
    clear_output(True)
    display(anz_restriktionen) 
    display(anz_var)
    display(anz_schlupf_var)
    display(anz_kuenstl_var)
    display(button)
    
def start_simplex(button):
    copy_tableau = ipysheet.to_dataframe(input_table)
    copy_tableau = copy_tableau.apply(pd.to_numeric, errors='ignore', downcast='float')
    copy_tableau.columns = range(0, sum_var+3)
    cj = []
    cj_zj = []
    for column in range(0,len(copy_tableau.columns)):
        if column == 0:
            cj.append(np.nan)
            cj_zj.append(np.nan)
        elif column == 1:
            cj.append("cj")
            cj_zj.append("cj-zj")
        elif column == 2:
            cj.append(0)
            cj_zj.append(np.nan)
        else:
            cj.append(0)
            cj_zj.append(0)
            
    copy_tableau.loc[len(copy_tableau.index)] = cj 
    copy_tableau.loc[len(copy_tableau.index)] = cj_zj 
    copy_tableau.replace('-M', -M,  inplace = True)
    
    for row in copy_tableau.index:
        for column in copy_tableau.columns:
            try: 
                copy_tableau.loc[row][column] = int(copy_tableau.loc[row][column])
            except Exception:
                pass
    
    sa.get_cj_zj(copy_tableau)
    global tableau 
    tableau = copy_tableau
    print(copy_tableau)
            

def adjust_cj(change):
    for row in range(2, len(cell_table)):
        if change['new'] == cell_table[row][1].value:
            if "s" in cell_table[row][1].value:
                cell_table[row][0].value = cell(row,0, 0, read_only=True, background_color = "white")
            elif "x" in cell_table[row][1].value:
                cell_table[row][0] = cell(row, 0, "...", background_color = 'yellow')

def update_table(table):
    input_table = table['new']
    simplex_start.disabled = True
    
    
    



## Definiere Art/Anzahl der Variablen

In [5]:
out = widgets.Output()



anz_restriktionen = widgets.Dropdown(
                        options=list(map(str, range(0, 6))),
                        value= '0',
                        description='Restriktionen:',
                        disabled=False,
                    )

anz_var = widgets.Dropdown(
            options=list(map(str, range(1, 6))),
            value='1',
            description='echte Variablen:',
            disabled=False,
         )

anz_schlupf_var = widgets.Dropdown(
                    options=list(map(str, range(0, 6))),
                    value='0',
                    description='Schlupfvariablen:',
                    disabled=False,
                )

anz_kuenstl_var = widgets.Dropdown(
                    options=list(map(str, range(0, 6))),
                    value='0',
                    description='künstliche Variablen:',
                    disabled=False,
                )
sum_var=int(anz_var.value) + int(anz_schlupf_var.value) + int(anz_kuenstl_var.value)
button = widgets.Button(description="Erzeuge Eingabetableau!")




anz_restriktionen.observe(on_restr_change, names='value')
button.on_click(create_tableau)
display(out)

anz_restriktionen.layout.visibility = "visible"
anz_var.layout.visibility = "hidden"
anz_schlupf_var.layout.visibility = "hidden"
anz_kuenstl_var.layout.visibility = "hidden"
button.layout.visibility = "hidden"

with out:
    display(anz_restriktionen) 
    display(anz_var)
    display(anz_schlupf_var)
    display(anz_kuenstl_var)
    display(button)
    
    
    


Output()

## Erzeuge das Standardtableau

In [ ]:
%blockrun button

In [ ]:
list_var = []

spalte = 3
reihe_basis_var = 2
sum_var=int(anz_var.value) + int(anz_schlupf_var.value) + int(anz_kuenstl_var.value)
input_table = ipysheet.sheet(rows=2+float(anz_restriktionen.value), columns=sum_var+3,  row_headers=False, column_headers=False)
M = sp.symbols('M')
tableau = "globale Variable"


    
    
    
    

    
#zwei Dimensionales Array, welches Sheet repräsentiert
cell_table = [[0]*input_table.columns for i in range(input_table.rows)]

for row in range(0,input_table.rows):
    for column in range(0,input_table.columns):
        if column != 0 and column != 1:
            cell_table[row][column] = cell(row, column, "...", background_color = 'yellow')
            cell_table[row][column].observe(update_table)


cell_table[0][0] = cell(0,0, "", read_only=True, background_color='grey')
cell_table[0][1] = cell(0,1, "", read_only=True, background_color='grey')
cell_table[0][2] = cell(0,2, "", read_only=True, background_color='grey')

#Befülle Reihe mit Beschreibung

cell_table[1][0] = cell(1, 0, "cj", read_only=True, font_weight = 'bold', background_color = "white")
cell_table[1][1] = cell(1, 1, "Basisvariable", read_only=True, font_weight = 'bold', background_color = "white")
cell_table[1][2] = cell(1, 2, "Quantity", read_only=True, font_weight = 'bold', background_color = "white")

for anz in range(1, int(anz_var.value)+1):
    var_name = "x"+ str(anz)
    cell_table[1][spalte] = cell(1,spalte, var_name, ready_only=True,font_weight = 'bold', background_color = "white")
    list_var.append(cell_table[1][spalte])
    spalte += 1

for anz in range(1, int(anz_schlupf_var.value)+1):
    var_name = "s"+ str(anz)
    cell_table[0][spalte] = cell(0,spalte, 0, read_only=True, background_color = "white")
    cell_table[1][spalte] = cell(1,spalte, var_name, read_only=True, font_weight = 'bold', background_color = "white")   
    list_var.append(cell_table[1][spalte])
    spalte += 1

for anz in range(1, int(anz_kuenstl_var.value)+1):
    var_name =  "a"+ str(anz)
    cell_table[0][spalte] = cell(0,spalte, "-M", read_only=True, background_color = "white")
    cell_table[1][spalte] = cell(1,spalte, var_name, read_only=True, font_weight = 'bold', background_color = "white")   
    list_var.append(cell_table[1][spalte])
    spalte += 1

start_basisvar = list_var[(len(list_var)-int(anz_restriktionen.value)):]
start_basisvar.sort(key=lambda x: x.value)
basis_selection = []

for var in list_var:
    if "a" in var.value:
        break
    basis_selection.append(var.value)
    
    


    
  
    

for row in range(2,input_table.rows):
    if "a" in start_basisvar[row-2].value:
        cell_table[row][0] = cell(row,0, '-M', read_only=True, background_color = "white")
        cell_table[row][1] = cell(row,1, start_basisvar[row-2].value, read_only=True, font_weight = 'bold', background_color = "white")
    if "s" in start_basisvar[row-2].value:
        cell_table[row][0] = cell(row,0, 0, read_only=True, background_color = "white")  
        cell_table[row][1] = cell(row,1, start_basisvar[row-2].value, read_only=False, font_weight = 'bold', choice = basis_selection, background_color = "yellow")
        cell_table[row][1].observe(adjust_cj)
    if "x" in start_basisvar[row-2].value:
        cell_table[row][1] = cell(row,1, start_basisvar[row-2].value, read_only=False, font_weight = 'bold', choice = basis_selection, background_color = "yellow")
        cell_table[row][0] = cell(row, 0, "...", background_color = 'yellow')
        cell_table[row][1].observe(adjust_cj)
    

def correct_input(button):
    wrong_input_counter = 0
    for row in range(0,input_table.rows):
        for column in range(0,input_table.columns):
            if cell_table[row][column].style == {'backgroundColor': 'yellow'} or cell_table[row][column].style == {'backgroundColor': 'red'} and column != 1:
                try:
                    test = float(cell_table[row][column].value)
                    cell_table[row][column].style = {'backgroundColor': 'yellow'}
                except ValueError:
                    cell_table[row][column].style = {'backgroundColor': 'red'}
                    wrong_input_counter += 1
    if wrong_input_counter == 0:
        simplex_start.disabled = False
    else:
        simplex_start.disabled = True
                


display(input_table)

simplex_start = widgets.Button(description="Starte Simplex", disabled=True)
check_input = widgets.Button(description="Überprüfe Eingabe")
check_input.on_click(correct_input)
display(check_input)
display(simplex_start)

 

simplex_start.on_click(start_simplex)

# example
#0                                      300         200   0   0  -M  -M
#1   cj  Basisvariable  Quantity         x1          x2  s1  s2  a1  a2
#2   -M             a1        60          2           2   0   0   1   0
#3   -M             a2        80          2           8  -1   0   0   1
#4    0             s2        40          1           0   0   1   0   0
#5  NaN             cj    -140*M       -4*M       -10*M   M   0  -M  -M
#6  NaN          cj-zj       NaN  4*M + 300  10*M + 200  -M   0   0   0

In [ ]:
%blockrun simplex_start

In [ ]:
#Simplex Algorithmus
list_tableaus, Meldungen, list_pivot_elements = sa.simplex_algorithm(tableau, 10,M)

## Ergebnis 

In [ ]:
pd.set_option("precision", 3)



#Erzeuge Tableaus (untereinander damit man es ausdrucken kann)
for table in range(0,len(list_tableaus)):
    display(widgets.Label(value= str(table) + ".Tableau"))
    #display(list_tableaus[table].style.hide_index().hide_columns())
    display(list_tableaus[table].style\
                .apply(lambda x: ['background: lightblue' if x.name == list_pivot_elements[table][1] else '' for i in x])\
                .apply(lambda x: ['background: lightblue' if x.name == list_pivot_elements[table][0] else '' for i in x], axis=1)\
                .hide_index()\
                .hide_columns())    
    for message in range(len(Meldungen[table])):
          display(widgets.Label(value=Meldungen[table][message]))
    print()

    

            
